### Before combining all data, I want to check how many matches to the Netflix data we can get with the Rotten Tomatoes and Letterboxd data, separately.

Below are the fields of the tables saved in the database.

### netflix – Contains the Netflix original film list (webscraped from Wikipedia):
- title (string) - official Netflix title
- releaseDate (string) - release date in '{month} {day}, {year}' format (e.g. 'October 16, 2015') 
- genre - main genre
- runtime - runtime of the flim in '{hour} h {minute} min' format (e.g. '1 h 40 min')
- language - main langauge of the film
- filmType - type of the film (e.g. feature film, documentary, special, etc)
- runtime_min - runtime converted to minutes

### rotten_tomatoes
- id (string) - Unique identifier for each movie
- title - The title of the movie
- audienceScore - The average score given by regular viewers (0-100)
- tomatoMeter - The percentage of positive reviews from professional critics (0-100)
- rating - The movie's age-based classification (e.g., G, PG, PG-13, R)
- ratingContents - Content leading to the rating classification
- releaseDateTheaters - The date the movie was released in theaters
- releaseDateStreaming - The date the movie became available for streaming
- runtimeMinutes - The duration of the movie in minutes
- genre - The movie's genre(s)
- originalLanguage - The original language of the movie
- director - The movie's director
- writer - The writer(s) responsible for the movie's screenplay
- boxOffice - The movie's total box office revenue
- distributer - The company responsible for distributing the movie
- soundMix - The audio format(s) used in the movie
- releaseYearEarlier - The year the movie was released, either in theaters or on a streaming service, whichever is earlier

### letterboxd
- id - unique identifier for each movie
- name - title of the movie
- date - release year of the movie
- minute - runtime in minutes
- rating - rating on letterboxd

In [1]:
import pandas as pd
import os
import sqlite3 

# Connect to SQLite database 
conn = sqlite3.connect('films.db') 

# Create a cursor object 
cur = conn.cursor() 

In [2]:
# Check if rotten_tomatoes dataframe was saved correctly by loading and pulling up first 10 entries
query = """
        SELECT *
        FROM rotten_tomatoes
        LIMIT 10;
        """
# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)
for row in out: 
    print(row) 

['id', 'title', 'audienceScore', 'tomatoMeter', 'rating', 'ratingContents', 'releaseDateTheaters', 'releaseDateStreaming', 'runtimeMinutes', 'genre', 'originalLanguage', 'director', 'writer', 'boxOffice', 'distributor', 'soundMix', 'releaseYearEarlier']
('space-zombie-bingo', 'Space Zombie Bingo!', 50, None, None, None, None, '2018-08-25', 75, 'Comedy, Horror, Sci-fi', 'English', 'George Ormrod', 'George Ormrod,John Sabotta', None, None, None, 2018.0)
('love_lies', 'Love, Lies', 43, None, None, None, None, None, 120, 'Drama', 'Korean', 'Park Heung-Sik,Heung-Sik Park', 'Ha Young-Joon,Jeon Yun-su,Song Hye-jin', None, None, None, None)
('the_sore_losers_1997', 'Sore Losers', 60, None, None, None, None, '2020-10-23', 90, 'Action, Mystery & thriller', 'English', 'John Michael McCarthy', 'John Michael McCarthy', None, None, None, 2020.0)
('dinosaur_island_2002', 'Dinosaur Island', 70, None, None, None, None, '2017-03-27', 80, 'Fantasy, Adventure, Animation', 'English', 'Will Meugniot', 'John

Let's try joining netflix and rotten_tomatoes table on the title. Starting with the least constraint to see how many matches just based on title we can get.

In [33]:
# Joining on title, year, and language and checking how many non-unique entries we get
query = """
        SELECT *
        FROM netflix AS n
        INNER JOIN rotten_tomatoes AS r
        ON LOWER(n.title) = LOWER(r.title) AND n.releaseYear = r.releaseYearEarlier AND n.language = r.originalLanguage
        GROUP BY n.id
        HAVING COUNT(*) > 1;
        """

# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)

for row in out: 
    print(row) 

['id', 'title', 'releaseDate', 'genre', 'runtime', 'language', 'filmType', 'runtime_min', 'releaseYear', 'id', 'title', 'audienceScore', 'tomatoMeter', 'rating', 'ratingContents', 'releaseDateTheaters', 'releaseDateStreaming', 'runtimeMinutes', 'genre', 'originalLanguage', 'director', 'writer', 'boxOffice', 'distributor', 'soundMix', 'releaseYearEarlier']
(132, 'Cargo', 'May 18, 2018', 'Drama / Horror', '1 h 44 min', 'English', 'Feature films', 104, 2018, 'cargo_2017', 'Cargo', None, 60, None, None, '2019-05-30', '2018-02-09', 112, 'Drama', 'English', 'Kareem Mortimer', 'Kareem Mortimer', None, 'Artist Rights Distribution', None, 2018.0)
(218, 'Triple Frontier', 'March 13, 2019', 'Action-adventure', '2 h 5 min', 'English', 'Feature films', 125, 2019, 'triple_frontier', 'Triple Frontier', 55, 70, 'R', "['Language Throughout', 'Violence']", '2019-03-06', '2019-03-15', 125, 'Action, Adventure, Mystery & thriller', 'English', 'J.C. Chandor', 'Mark Boal,J.C. Chandor', None, 'Netflix', None,

In [35]:
# checking one entry from the output above in the rotten_tomatoes table
query = """
        SELECT *
        FROM rotten_tomatoes
        WHERE id = 'the_highwaymen';
        """

# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)

for row in out: 
    print(row) 

['id', 'title', 'audienceScore', 'tomatoMeter', 'rating', 'ratingContents', 'releaseDateTheaters', 'releaseDateStreaming', 'runtimeMinutes', 'genre', 'originalLanguage', 'director', 'writer', 'boxOffice', 'distributor', 'soundMix', 'releaseYearEarlier']
('the_highwaymen', 'The Highwaymen', 74, 58, 'R', "['Bloody Images', 'Some Strong Violence']", '2019-03-29', '2019-03-30', 132, 'Crime, Drama', 'English', 'John Lee Hancock', 'John Fusco', None, 'Netflix', None, 2019.0)
('the_highwaymen', 'The Highwaymen', 74, 58, 'R', "['Bloody Images', 'Some Strong Violence']", '2019-03-29', '2019-03-30', 132, 'Crime, Drama', 'English', 'John Lee Hancock', 'John Fusco', None, 'Netflix', None, 2019.0)


Seems like there are entries with the same id in rotten_tomatoes. Going to group by the id first and then join to prevent duplicates. Adding in the runtime constraint to make sure we are getting "close to" precise matches as much as possible.

In [23]:
query = """
        SELECT COUNT( DISTINCT n.id ) AS unique_id, COUNT(*) AS total
        FROM netflix AS n
        INNER JOIN (
            SELECT *
            FROM rotten_tomatoes 
            GROUP BY id) AS r
        ON LOWER(n.title) = LOWER(r.title) AND n.releaseYear = r.releaseYearEarlier AND n.language = r.originalLanguage
        WHERE ( r.runtimeMinutes <= n.runtime_min +5 AND r.runtimeMinutes >= n.runtime_min -5 );
        """

# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)

for row in out: 
    print(row) 

['unique_id', 'total']
(604, 604)


Moving onto letterboxd.

In [25]:
# Check if letterboxd dataframe was saved correctly by loading and pulling up first 10 entries
query = """
        SELECT *
        FROM letterboxd
        LIMIT 10;
        """
# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)
for row in out: 
    print(row) 

['id', 'name', 'date', 'minute', 'rating']
('1000001', 'Barbie', 2023, 114, 3.86)
('1000002', 'Parasite', 2019, 133, 4.56)
('1000003', 'Everything Everywhere All at Once', 2022, 140, 4.3)
('1000004', 'Fight Club', 1999, 139, 4.27)
('1000005', 'La La Land', 2016, 129, 4.09)
('1000006', 'Oppenheimer', 2023, 181, 4.23)
('1000007', 'Interstellar', 2014, 169, 4.35)
('1000008', 'Joker', 2019, 122, 3.85)
('1000009', 'Dune', 2021, 155, 3.9)
('1000010', 'Pulp Fiction', 1994, 154, 4.26)


Let's proactively check if there are entries with duplicate ids in the letterboxd table as in the rotten_tomatoes table.

In [29]:
query = """
        SELECT id, COUNT(id)
        FROM letterboxd
        GROUP BY id
        HAVING COUNT(id)>1;
        """
# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)
for row in out: 
    print(row) 

['id', 'COUNT(id)']


Seems like all entries have unique ids! Moving onto combining with netflix data.

In [37]:
# Joining on title/name and year/date, and checking for non-unique entries
query = """
        SELECT *
        FROM netflix AS n
        INNER JOIN letterboxd AS l
        ON LOWER(n.title) = LOWER(l.name) AND n.releaseYear = l.date
        GROUP BY n.id
        HAVING COUNT(*) > 1;
        """

# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)

for row in out: 
    print(row) 

['id', 'title', 'releaseDate', 'genre', 'runtime', 'language', 'filmType', 'runtime_min', 'releaseYear', 'id', 'name', 'date', 'minute', 'rating']
(94, 'One of Us', 'October 20, 2017', None, '1 h 35 min', 'English', 'Documentaries', 95, 2017, '1014825', 'One of Us', 2017, 95, 3.47)
(239, 'Beats', 'June 19, 2019', 'Coming-of-age drama', '1 h 50 min', 'English', 'Feature films', 110, 2019, '1010859', 'Beats', 2019, 100, 3.71)
(441, 'The Beast', 'November 27, 2020', 'Action thriller', '1 h 39 min', 'Italian', 'Feature films', 99, 2020, '1035031', 'The Beast', 2020, 90, 2.43)
(442, 'The Call', 'November 27, 2020', 'Psychological thriller', '1 h 52 min', 'Korean', 'Feature films', 112, 2020, '1002380', 'The Call', 2020, 112, 3.57)
(463, 'Becoming', 'May 6, 2020', None, '1 h 29 min', 'English', 'Documentaries', 89, 2020, '1011923', 'Becoming', 2020, 89, 3.63)
(771, 'Bubble', 'April 28, 2022', 'Anime', '1 h 41 min', 'Japanese', 'Feature films', 101, 2022, '1006558', 'Bubble', 2022, 101, 2.89)

Let's add in the runtime constraint.

In [49]:
# Adding runtime constraint and checking the number of unique entries
query = """
        SELECT COUNT( DISTINCT n.id ) AS unique_id, COUNT(*) AS total
        FROM netflix AS n
        INNER JOIN letterboxd AS l
        ON LOWER(n.title) = LOWER(l.name) AND n.releaseYear = l.date
        WHERE ( l.minute <= n.runtime_min +5 AND l.minute >= n.runtime_min -5 );
        """

# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)

for row in out: 
    print(row) 

['unique_id', 'total']
(1109, 1110)


In [43]:
# Check the entry in netflix and letterboxd
query = """
        SELECT *
        FROM netflix
        WHERE title = 'Bubble';
        """
# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)
for row in out: 
    print(row) 
    
query = """
        SELECT *
        FROM letterboxd
        WHERE name = 'Bubble';
        """
# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)
for row in out: 
    print(row) 

['id', 'title', 'releaseDate', 'genre', 'runtime', 'language', 'filmType', 'runtime_min', 'releaseYear']
(771, 'Bubble', 'April 28, 2022', 'Anime', '1 h 41 min', 'Japanese', 'Feature films', 101, 2022)
['id', 'name', 'date', 'minute', 'rating']
('1006558', 'Bubble', 2022, 101, 2.89)
('1012729', 'Bubble', 2005, 74, 3.4)
('1085051', 'Bubble', 2022, 103, 3.44)


TODO: maybe would be good to include primary language in letterboxd table and use as a condition to join as well?

In [50]:
# Load CSV file into a DataFrame\
path = '/Users/sunyoungpark/.cache/kagglehub/datasets/gsimonx37/letterboxd/versions/2/languages.csv'
df = pd.read_csv(path)
df = df.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1038762 entries, 0 to 1038761
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   id        1038762 non-null  Int64 
 1   type      1038762 non-null  string
 2   language  1038762 non-null  string
dtypes: Int64(1), string(2)
memory usage: 24.8 MB


In [51]:
# Write the data to a sqlite table 
df.to_sql('letterboxd_languages', conn, if_exists='replace', index=False) 

1038762

In [54]:
query = """
        SELECT l.id, l.name, l.date, l.minute, l.rating, lang.language
        FROM letterboxd AS l
        LEFT JOIN letterboxd_languages AS lang
        ON l.id = lang.id
        WHERE lang.type = 'Primary language'
        LIMIT 10;
        """
# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)
for row in out: 
    print(row) 

['id', 'name', 'date', 'minute', 'rating', 'language']
('1000002', 'Parasite', 2019, 133, 4.56, 'Korean')
('1000003', 'Everything Everywhere All at Once', 2022, 140, 4.3, 'English')
('1000006', 'Oppenheimer', 2023, 181, 4.23, 'English')
('1000009', 'Dune', 2021, 155, 3.9, 'English')
('1000010', 'Pulp Fiction', 1994, 154, 4.26, 'English')
('1000011', 'Spider-Man: Into the Spider-Verse', 2018, 117, 4.42, 'English')
('1000015', 'Knives Out', 2019, 131, 3.99, 'English')
('1000016', 'Midsommar', 2019, 147, 3.78, 'English')
('1000017', 'The Dark Knight', 2008, 152, 4.47, 'English')
('1000018', 'Inception', 2010, 148, 4.2, 'English')


Now using this joined table as a subquery to join to the netflix table

In [55]:
# Adding language match as another join condition and checking the number of unique entries
query = """
        SELECT COUNT( DISTINCT n.id ) AS unique_id, COUNT(*) AS total
        FROM netflix AS n
        INNER JOIN (
            SELECT l.id, l.name, l.date, l.minute, l.rating, lang.language
            FROM letterboxd AS l
            LEFT JOIN letterboxd_languages AS lang
            ON l.id = lang.id
            WHERE lang.type = 'Primary language'
            ) AS l
        ON LOWER(n.title) = LOWER(l.name) AND n.releaseYear = l.date AND n.language = l.language;
        """
# WHERE ( l.minute <= n.runtime_min +5 AND l.minute >= n.runtime_min -5 )
# print column names
out = cur.execute(query)
names = list(map(lambda x: x[0], cur.description))
print(names)

for row in out: 
    print(row) 

KeyboardInterrupt: 

In [56]:
# Close connection to SQLite database 
conn.close()